# infrared and Visible fused images Super-Resolution module Notebook simplified


This notebook processes all images in a specified folder sequentially, one image at a time at full resolution, with optimized memory management for Google Colab T4 GPU.

## 1. SETUP CELL - Run First

In [1]:
# Mount Google Drive and import libraries
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
from IPython.display import display
import glob
import gc


# Set PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Mount Google Drive
drive.mount('/content/drive')

# Verify GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "No GPU available")

Mounted at /content/drive
Using device: cuda
GPU: Tesla T4


## 2. MODEL DEFINITION CELL

In [8]:
class ResidualDenseBlock(nn.Module):
    def __init__(self, channels, growth_channels=32):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, growth_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels + growth_channels, growth_channels, 3, padding=1)
        self.conv3 = nn.Conv2d(channels + 2*growth_channels, growth_channels, 3, padding=1)
        self.conv4 = nn.Conv2d(channels + 3*growth_channels, channels, 3, padding=1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat([x, x1], 1)))
        x3 = self.lrelu(self.conv3(torch.cat([x, x1, x2], 1)))
        x4 = self.conv4(torch.cat([x, x1, x2, x3], 1))
        return x + x4 * 0.2

class ReversedUBlock(nn.Module):
    def __init__(self, channels, scale_factor=2):
        super().__init__()
        self.up = nn.Sequential(
            nn.Conv2d(channels, channels * (scale_factor**2), 3, padding=1),
            nn.PixelShuffle(scale_factor),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.process = nn.Sequential(
            ResidualDenseBlock(channels),
            ResidualDenseBlock(channels)
        )
        self.down = nn.Sequential(
            nn.Conv2d(channels, channels, 3, stride=scale_factor, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1)
        )
        self.skip = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        x_up = self.up(x)
        x_proc = self.process(x_up)
        x_down = self.down(x_proc)
        return x_down + self.skip(x)

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c = x.shape[:2]
        y = self.gap(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * (1 + y)

class EnhancedSRUnet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, scale=4, num_rub=3):
        super().__init__()
        self.scale = scale
        self.head = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.rub_blocks = nn.ModuleList([
            nn.Sequential(
                ReversedUBlock(64),
                ChannelAttention(64),
                ResidualDenseBlock(64)
            ) for _ in range(num_rub)
        ])
        self.fusion = nn.Sequential(
            nn.Conv2d(64*(num_rub+1), 64, 1),
            nn.LeakyReLU(0.2, inplace=True),
            ChannelAttention(64)
        )
        self.upsample = nn.Sequential(
            nn.Conv2d(64, 64 * (scale**2), 3, padding=1),
            nn.PixelShuffle(scale),
            nn.Conv2d(64, out_channels, 3, padding=1)
        )
        self.refine = nn.Sequential(
            nn.Conv2d(out_channels, 32, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, out_channels, 3, padding=1)
        )

    def forward(self, x):
        x = x.to(next(self.parameters()).device)
        x0 = self.head(x)
        features = [x0]
        for rub in self.rub_blocks:
            features.append(rub(features[-1]))
        x_fused = self.fusion(torch.cat(features, dim=1))
        out = self.upsample(x_fused)
        out = out + F.interpolate(x, scale_factor=self.scale, mode='bicubic')
        return self.refine(out).clamp_(0, 1)

## 3. PATH SETUP AND MODEL LOAD CELL

In [9]:
# Define paths
model_path = '//'
input_folder = '//'
output_folder = '//'
excel_path = '//'

# Create directories if they don't exist
os.makedirs(os.path.dirname(model_path), exist_ok=True)
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Get list of all image files in the input folder
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(input_folder, ext)))
print(f"Found {len(image_paths)} images to process")

# Load model once
model = EnhancedSRUnet().to(device)
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    print(f"Expected model at: {model_path}")
    model = None

## 4. IMAGE PROCESSING FUNCTION

In [10]:
def process_all_images():
    if model is None:
        print("Model not loaded. Cannot process images.")
        return None

    model.eval()
    processed_images = []

    # Process each image sequentially
    for img_path in image_paths:
        print(f"\n=== Processing image: {os.path.basename(img_path)} ===")

        # Clear GPU memory before processing
        torch.cuda.empty_cache()
        gc.collect()

        # Load image
        try:
            img = Image.open(img_path).convert('RGB')
            print(f"Original image size: {img.size}")
        except Exception as e:
            print(f"Error loading image {os.path.basename(img_path)}: {str(e)}")
            continue

        # Convert to tensor
        img_tensor = torch.from_numpy(np.array(img)/255.).permute(2,0,1).unsqueeze(0).float().to(device)

        # Memory check
        required_memory = img_tensor.nelement() * img_tensor.element_size() * 20  # Rough estimate
        total_memory = torch.cuda.get_device_properties(0).total_memory if torch.cuda.is_available() else 0
        if torch.cuda.is_available() and required_memory > total_memory:
            print(f"Warning: Image {os.path.basename(img_path)} too large for GPU memory (needs ~{required_memory/1024**3:.1f}GB, available ~{total_memory/1024**3:.1f}GB)")
            del img_tensor
            torch.cuda.empty_cache()
            gc.collect()
            continue

        # Process entire image
        with torch.no_grad():
            try:
                output = model(img_tensor)
                print(f"Processing completed for {os.path.basename(img_path)}")
            except RuntimeError as e:
                print(f"GPU Memory Error for {os.path.basename(img_path)}: {str(e)}")
                print("Image size too large for T4 GPU. Consider using a higher-capacity GPU or reducing image size.")
                del img_tensor
                torch.cuda.empty_cache()
                gc.collect()
                continue

        # Convert and save output
        output_np = output.squeeze().permute(1,2,0).clamp(0,1).cpu().numpy()
        output_img = Image.fromarray((output_np*255).astype('uint8'))

        # Generate output path
        output_filename = os.path.splitext(os.path.basename(img_path))[0] + '_sr.png'
        output_path = os.path.join(output_folder, output_filename)

        # Save with maximum quality
        output_img.save(output_path, quality=100, subsampling=0)
        print(f"Super-resolved image saved to: {output_path}")

        processed_images.append((output_img, img_path, output_path))

        # Clear GPU memory after processing
        del img_tensor, output
        torch.cuda.empty_cache()
        gc.collect()

    return processed_images

In [5]:
# 6. BATCH IMAGE PROCESSING FUNCTION  (for Limitd GPU Memory)
from scipy.ndimage import gaussian_filter

def post_process_image(image):
    img_np = np.array(image).astype(np.float32)
    blurred = gaussian_filter(img_np, sigma=1)
    sharpened = img_np + (img_np - blurred) * 1.5
    sharpened = np.clip(sharpened, 0, 255).astype(np.uint8)
    return Image.fromarray(sharpened)

def process_all_images():
    # Load model with self-ensemble
    model = EnhancedSRUnet().to(device)
    model = SelfEnsembleWrapper(model)
    try:
        state_dict = torch.load(model_path, map_location=device)
        model.model.load_state_dict(state_dict)
        print("Model loaded successfully with self-ensemble")
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        print(f"Expected model at: {model_path}")
        return None

    # Get list of images
    image_extensions = ['*.jpg', '*.jpeg', '*.png']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_folder, ext)))

    if not image_paths:
        print(f"No images found in {input_folder}")
        return None

    # Initialize results list
    results = []

    model.eval()
    for img_path in image_paths:
        print(f"\nProcessing: {os.path.basename(img_path)}")

        # Load image
        try:
            img = Image.open(img_path).convert('RGB')
            print(f"Original image size: {img.size}")
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            continue

        # Process image with chunking
        output_img = process_image_in_chunks(model, img, device, chunk_size=512, overlap=64)
        if output_img is None:
            print(f"Failed to process {img_path}")
            continue

        # Apply post-processing
        output_img = post_process_image(output_img)

        # Save output
        output_filename = f"sr_{os.path.basename(img_path)}"
        output_img_path = os.path.join(output_folder, output_filename)
        output_img.save(output_img_path, quality=100, subsampling=0)
        print(f"Super-resolved image saved to: {output_img_path}")

        # Calculate metrics
        bicubic_img = img.resize((img.size[0]*4, img.size[1]*4), Image.BICUBIC)
        metrics = calculate_metrics(bicubic_img, output_img)
        metrics['Image'] = os.path.basename(img_path)
        results.append(metrics)

        # Clear memory
        del output_img, bicubic_img
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    # Save results to Excel
    if results:
        df = pd.DataFrame(results)
        df.to_excel(excel_path, index=False)
        print(f"\nMetrics saved to: {excel_path}")
    else:
        print("\nNo images were processed successfully.")

    return results

In [6]:
!pip install -U scikit-image


## 5. EXECUTION CELL - Run Last

In [11]:
print("\n=== Starting Image Processing ===")
results = process_all_images()
if results:
    print("\n=== Processing Summary ===")
    for output_img, input_path, output_path in results:
        print(f"\nImage: {os.path.basename(input_path)}")
        print(f"Input dimensions: {Image.open(input_path).size}")
        print(f"Output dimensions: {output_img.size}")
        display(output_img)
else:
    print("\nProcessing failed or no images found. Check error messages above.")

## 6. VERIFICATION CELL (Run if needed)

In [ ]:
print("\n=== File Verification ===")
print("Model exists:", os.path.exists(model_path))
print("Input directory exists:", os.path.exists(input_folder))
print("Output directory exists:", os.path.exists(output_folder))
print("Number of input images found:", len(image_paths))